In [1]:
!pip install firebase_admin

  Using cached pyparsing-3.2.0-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.6/12.6 MB 8.7 MB/s eta 0:00:00a 0:00:01
Using cached pyparsing-3.2.0-py3-none-any.whl (106 kB)
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-tools 1.62.3 requires protobuf<5.0dev,>=4.21.6, but you have protobuf 5.29.1 which is incompatible.
opentelemetry-proto 1.27.0 requires protobuf<5.0,>=3.19, but you have protobuf 5.29.1 which is incompatible.


# JSON with all the Data

In [ ]:
import firebase_admin
from firebase_admin import credentials, firestore
import json

# Initialize Firebase Admin SDK
firebase_cred_path = "perkcup-689f9-firebase-adminsdk-yuf7v-c17798a18e.json"  # Replace with your Firebase Admin JSON
if not firebase_admin._apps:
    cred = credentials.Certificate(firebase_cred_path)
    firebase_admin.initialize_app(cred)

# Firestore client
db = firestore.client()

def fetch_all_documents(collection_ref):
    """Recursively fetch all documents and their subcollections."""
    data = {}
    try:
        for doc in collection_ref.stream():
            doc_id = doc.id
            print('doc :' , doc_id)
            doc_data = doc.to_dict()
            data[doc_id] = doc_data

            # Check for subcollections
            subcollections = db.collection(collection_ref.id).document(doc_id).collections()
            for subcollection in subcollections:
                subcollection_name = subcollection.id
                data[doc_id][subcollection_name] = fetch_all_documents(subcollection)

        return data
    except Exception as e:
        print(f"Error fetching documents: {e}")
        return {}

def fetch_full_database():
    """Fetch the full Firestore database content."""
    database_content = {}
    try:
        # Fetch all top-level collections
        collections = db.collections()
        for collection in collections:
            collection_name = collection.id
            print(collection_name)
            database_content[collection_name] = fetch_all_documents(collection)
        return database_content
    except Exception as e:
        print(f"Error fetching database: {e}")
        return {}

import json
from datetime import datetime

def save_to_file(data, filename):
    """Save the fetched data to a JSON file."""
    def default_serializer(obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # Convert datetime to ISO 8601 string
        if hasattr(obj, '__dict__'):
            return obj.__dict__  # Serialize objects with `__dict__` attribute
        return str(obj)  # Fallback to string representation

    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4, default=default_serializer)
        print(f"Database content saved to {filename}")
    except Exception as e:
    
        print(f"Error saving to file: {e}")

if __name__ == "__main__":
    print("Fetching full Firestore database content...")
    full_database_content = fetch_full_database()
    print("Saving...")
    # Save the fetched data to a JSON file
    save_to_file(full_database_content, "firestore_full_database.json")

# Blocks

In [8]:
import firebase_admin
from firebase_admin import credentials, firestore
import json
from datetime import datetime

# Initialize Firebase Admin SDK
firebase_cred_path = "perkcup-689f9-firebase-adminsdk-yuf7v-c17798a18e.json"  # Replace with your Firebase Admin JSON
if not firebase_admin._apps:
    cred = credentials.Certificate(firebase_cred_path)
    firebase_admin.initialize_app(cred)

# Firestore client
db = firestore.client()

def fetch_blocks_collection():
    """Fetch all documents in the 'blocks' collection."""
    blocks_data = {}
    try:
        blocks_collection = db.collection('blocks')
        for doc in blocks_collection.stream():
            doc_id = doc.id
            print(f"Fetching document: {doc_id}")
            try:
                doc_data = doc.to_dict()
                blocks_data[doc_id] = doc_data
                
                # Fetch subcollections for each document
                subcollections = db.collection('blocks').document(doc_id).collections()
                for subcollection in subcollections:
                    subcollection_name = subcollection.id
                    print(f"Fetching subcollection: {subcollection_name}")
                    blocks_data[doc_id][subcollection_name] = {
                        sub_doc.id: sub_doc.to_dict() for sub_doc in subcollection.stream()
                    }
            except Exception as e:
                print(f"Error processing document {doc_id}: {e}")

        return blocks_data
    except Exception as e:
        print(f"Error fetching 'blocks' collection: {e}")
        return {}

def save_to_file(data, filename):
    """Save the fetched data to a JSON file."""
    def default_serializer(obj):
        if isinstance(obj, datetime):
            return obj.isoformat()  # Convert datetime to ISO 8601 string
        return str(obj)  # Fallback to string representation

    try:
        with open(filename, 'w') as file:
            json.dump(data, file, indent=4, default=default_serializer)
        print(f"Database content saved to {filename}")
    except Exception as e:
        print(f"Error saving to file: {e}")

if __name__ == "__main__":
    print("Fetching 'blocks' collection content...")
    blocks_content = fetch_blocks_collection()
    print("Saving...")
    save_to_file(blocks_content, "blocks_collection.json")

Fetching 'blocks' collection content...
Fetching document: 1
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 2
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 3
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 4
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 5
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 6
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 7
Fetching subcollection: perkcup_points
Fetching subcollection: players
Fetching subcollection: tournaments
Fetching document: 8
Fetching subcollection: perkcup_points
Fetc

In [7]:
import pandas as pd
import json

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)
file_path = "firestore_full_database.json"
x = load_json(file_path)
x["players"]

{'10081': {'player_name': 'Bae, Sangmoon',
  'dg_rank': 444,
  'last_updated': '2025-01-25T17:42:20.149000+00:00',
  'country': 'KOR',
  'dg_index': -0.983544510876086,
  'hidden': True,
  'primary_tour': 'PGA',
  'owgr_rank': 501},
 '10091': {'cf_short_comp': nan,
  'sample_size': nan,
  '100_150_fw_gir_rate': 0.799,
  'over_150_rgh_sg_per_shot': 0.081,
  '100_150_fw_good_shot_rate': 0.08,
  'player_name_rank': 'McIlroy, Rory',
  '100_150_fw_sg_per_shot': 0.021,
  'dg_index': 1.98136860407427,
  'std_deviation': nan,
  'over_200_fw_poor_shot_avoid_rate': 0.947,
  'country_y': nan,
  'comments': 'yyyyy',
  'owgr_rank': 3,
  '50_100_fw_low_data_indicator': 0,
  'perkcuppoints2020': 605,
  'draft1': 14,
  'true_sg_adjustments': nan,
  'under_150_rgh_sg_per_shot': 0.031,
  'course_experience_adjustment': nan,
  'over_150_rgh_good_shot_rate': 0.089,
  'other_fit_adjustment': nan,
  '150_200_fw_shot_count': 716,
  'under_150_rgh_shot_count': 285,
  'over_200_fw_shot_count': 360,
  'dg_id_x'

In [24]:
import pandas as pd
import json

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

def extract_tournaments_data(data):
    """
    Extract tournaments data from the loaded JSON, including details about players and other event information.
    """
    tournaments_data = []
    for year, tours in data.get('tournaments', {}).items():
        for tour, events in tours.items():
            for event_id, event_details in events.items():
                # General tournament details
                tournament_info = {
                    'year': year,
                    'tour': tour,
                    'event_name': event_details.get('event_name'),
                    'event_id': event_id,
                    'isUpcoming': event_details.get('isUpcoming'),
                    'finish_date': event_details.get('finish_date'),
                    'last_fetched': event_details.get('last_fetched')
                }
                
                # Include players' data if available
                players = event_details.get('website_info', {}).get('players', [])
                for player in players:
                    player_info = {
                        'player_name': player.get('Name'),
                        'finish_position': player.get('Finish_Position'),
                        'perkcup_points': player.get('PerkCup_Value'),
                        'sg_total': player.get('SG_Total'),
                        'sg_app': player.get('SG_APP'),
                        'sg_ott': player.get('SG_OTT'),
                        'sg_putt': player.get('SG_PUTT'),
                        'sg_arg': player.get('SG_ARG'),
                        'field': player.get('Field'),
                        'comment': player.get('Comment')
                    }
                    # Combine tournament info with player info
                    tournaments_data.append({**tournament_info, **player_info})
                    
                # If no players, just append the tournament info
                if not players:
                    tournaments_data.append(tournament_info)

    return pd.DataFrame(tournaments_data)

def extract_players_data(data):
    """
    Extract players data from the loaded JSON.
    """
    players_data = []
    for year, tours in data.get('tournaments', {}).items():
        for tour, events in tours.items():
            for event_id, event_details in events.items():
                for player in event_details.get('Results', {}).get('scores', []):
                    player_data = {
                        'event_id': event_details.get('event_id'),
                        'player_name': player.get('player_name'),
                        'dg_id': player.get('dg_id'),
                        'final_position': player.get('fin_text')
                    }

                    # Add round details
                    for round_num in range(1, 5):
                        round_key = f'round_{round_num}'
                        round_details = player.get(round_key, {})
                        player_data.update({
                            f'round_{round_num}_score': round_details.get('score'),
                            f'round_{round_num}_sg_app': round_details.get('sg_app'),
                            f'round_{round_num}_course_par': round_details.get('course_par'),
                            f'round_{round_num}_driving_acc': round_details.get('driving_acc'),
                            f'round_{round_num}_teetime': round_details.get('teetime'),
                            f'round_{round_num}_sg_total': round_details.get('sg_total'),
                            f'round_{round_num}_sg_arg': round_details.get('sg_arg'),
                            f'round_{round_num}_gir': round_details.get('gir'),
                            f'round_{round_num}_sg_putt': round_details.get('sg_putt'),
                            f'round_{round_num}_prox_fw': round_details.get('prox_fw'),
                            f'round_{round_num}_scrambling': round_details.get('scrambling'),
                            f'round_{round_num}_course_num': round_details.get('course_num'),
                            f'round_{round_num}_great_shots': round_details.get('great_shots'),
                            f'round_{round_num}_course_name': round_details.get('course_name'),
                            f'round_{round_num}_sg_ott': round_details.get('sg_ott'),
                            f'round_{round_num}_start_hole': round_details.get('start_hole'),
                            f'round_{round_num}_prox_rgh': round_details.get('prox_rgh'),
                            f'round_{round_num}_poor_shots': round_details.get('poor_shots'),
                            f'round_{round_num}_driving_dist': round_details.get('driving_dist'),
                            f'round_{round_num}_sg_t2g': round_details.get('sg_t2g')
                        })
                    players_data.append(player_data)
    return pd.DataFrame(players_data)

def extract_players_info(data):
    """
    Extract players information from the loaded JSON.
    """
    players_data = data.get("players", {})
    return pd.DataFrame.from_dict(players_data, orient='index')

def save_to_csv(df, file_name):
    """
    Save a DataFrame to a CSV file.
    """
    df.to_csv(file_name, index=False)
    print(f"CSV file '{file_name}' has been created.")

def main1():
    # Load the JSON data
    data = load_json('firestore_full_database.json')

    # Extract tournaments and players data
    tournaments_df = extract_tournaments_data(data)
    #players_df = extract_players_data(data)
    players_info_df = extract_players_info(data)

    # Save to CSV
    # Define the required columns
    required_columns = [
        "year", "tour","finish_date", "event_name", "event_id", "last_fetched", 
        "player_name", "finish_position", "perkcup_points", 
        "sg_total", "sg_arg"
    ]

    # Filter the dataframe
    tournaments_df = tournaments_df[required_columns]  # Keep only required columns
    tournaments_df = tournaments_df[tournaments_df["tour"] == "pga"]  # Keep only rows where tour == 'pga'

    save_to_csv(tournaments_df, 'functions/CSV DATA/tournaments.csv')
    #save_to_csv(players_df, 'events.csv')
    save_to_csv(players_info_df, 'functions/CSV DATA/players_info.csv')


main1()
import json
import csv

# Load JSON data from file
with open("firestore_full_database.json", "r") as file:
    data = json.load(file)

# Extract relevant tournament data
tournaments = data.get("2025_tournaments", {})

tournament_rows = []

for tournament_id, tournament_data in tournaments.items():
    players = tournament_data.get("players", {})
    for player_id, player_data in players.items():
        tournament_rows.append([
            tournament_id,
            2025,
            player_data.get("player_name", ""),
            player_data.get("sg_total", 0),
            player_data.get("perkcup_points", 0),
            player_data.get("fin_text", "")
        ])

# Save to CSV
csv_filename = "functions/CSV DATA/2025_tournaments_data.csv"
with open(csv_filename, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["event_id", "year","player_name", "sg_total", "perkcup_points", "finish_position"])
    csv_writer.writerows(tournament_rows)

print(f"CSV file '{csv_filename}' created successfully.")
# Load both CSV files
df1 = pd.read_csv('functions/CSV DATA/2025_tournaments_data.csv')
df2 = pd.read_csv('functions/CSV DATA/tournaments.csv')
# Concatenate while keeping all columns from both DataFrames

combined_df = pd.concat([df1, df2], axis=0, ignore_index=True)
id_to_name = {
    6: "Sony Open in Hawaii",
    2: "The American Express",
    16: "The Sentry",
    4 : "Farmers Insurance Open"
}

# Fill missing event_name values based on event_id
combined_df['event_name'] = combined_df.apply(lambda row: id_to_name.get(row['event_id'], row['event_name']), axis=1)

# Save the merged DataFrame to a new CSV file
combined_df.to_csv('functions/CSV DATA/tournaments.csv', index=False)


import json
import pandas as pd

# Function to load JSON data
def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

# Load JSON file
file_path = "firestore_full_database.json"  # Replace with your file path
data = load_json(file_path)

# Extract scores data
scores = data.get("scores", {})

# Process all events
rows = []
for event_id, event_data in scores.items():
    years = event_data.get("years", {})
    for year, year_details in years.items():
        event_name = year_details.get("event_name", "N/A")
        tour = year_details.get("tour", "N/A")
        tournament_id = year_details.get("tournament_id", "N/A")
        date = year_details.get("date", "N/A")
        if date == "N/A":
            print(f"Missing date for event ID: {event_id}, year: {year}")
        if date != "N/A":
            print(f"Event ID: {event_id}, year: {year}, date: {date}")
        actual_results = year_details.get("actual_results", [])

        for player in actual_results:
            player_name = player.get("name", "N/A")
            total_score = player.get("total_score", 0)
            fin_text = player.get("fin_text", "N/A")
            strokes_gained = player.get("strokes_gained", 0)
            percup_value = player.get("perkcup_value", 0)  # Include `perkcup_value` with default 0

            # Add player details to rows
            rows.append({
                "Event ID": event_id,
                "Year": year,
                "Event Name": event_name,
                "Tour": tour,
                "Tournament ID": tournament_id,
                "Date": date,
                "Player Name": player_name,
                "Total Score": total_score,
                "Final Position": fin_text,
                "Strokes Gained": strokes_gained,
                "Perkcup Value": percup_value,
            })

# Create a DataFrame and save it as a CSV
output_file = "functions/CSV DATA/all_event_scores.csv"
df = pd.DataFrame(rows)
df.to_csv(output_file, index=False)
print(f"All event scores data saved to {output_file}")
import json
import csv

def load_json(file_path):
    """
    Load JSON data from a file.
    """
    with open(file_path, 'r') as file:
        return json.load(file)

import pandas as pd
import json

def save_blocks_with_names(blocks, output_file):
    # Extract PerkCup Points data
    perkcuppoints_data = []
    for block_key, block_data in blocks.items():
        for id_key, player_data in block_data.get("perkcup_points", {}).items():
            for year, points in player_data.items():
                year_cleaned = year  # Keep the year as-is
                perkcuppoints_data.append({"Block": block_key, "PlayerID": id_key, "Year": year_cleaned, "PerkCupPoints": points})
    
    # Convert to DataFrame and pivot years into columns
    perkcuppoints_df = pd.DataFrame(perkcuppoints_data)
    perkcuppoints_pivot = perkcuppoints_df.pivot(index=["Block", "PlayerID"], columns="Year", values="PerkCupPoints").reset_index()

    # Extract Players' Historical Finish Positions
    players_data = []
    for block_key, block_data in blocks.items():
        for player_name, player_details in block_data.get("players", {}).items():
            for year_tournament, position in player_details.get("historicalFinishPositions", {}).items():
                year, tournament = map(str, year_tournament.split(","))
                players_data.append({
                    "Block": block_key,
                    "PlayerName": player_name,
                    "Year": year,
                    "Tournament": tournament,
                    "Position": position,
                    "Participations": player_details.get("participations", 0),
                    "TotalFieldPoints": round(player_details.get("currentblocktotalfieldpoints", 0), 2)  # Format to 2 decimal places
                })

    # Convert to DataFrame
    players_df = pd.DataFrame(players_data)

    # Load Firestore player data for names and IDs
    firestore_file_path = 'firestore_full_database.json'
    with open(firestore_file_path, 'r') as f:
        firestore_data = json.load(f)
    
    # Extract player IDs and names
    players_ids_data = []
    for player_id, player_info in firestore_data.get("players", {}).items():
        players_ids_data.append({
            "PlayerID": player_id,
            "PlayerName": player_info.get("player_name", "Unknown")
        })
    players_ids_df = pd.DataFrame(players_ids_data)

    # Merge PerkCup Points with player IDs and names
    combined_perkcuppoints_df = pd.merge(perkcuppoints_pivot, players_ids_df, on="PlayerID", how="left")

    # Merge the combined PerkCup Points with players' data on PlayerName
    final_combined_df = pd.merge(players_df, combined_perkcuppoints_df, on="PlayerName", how="left")

    # Save the combined data to the specified output file
    final_combined_df.to_csv(output_file, index=False)
    print(f"Combined data saved to {output_file}")

# Example usage
# Load blocks data
blocks_file_path = 'blocks_collection.json'
with open(blocks_file_path, 'r') as f:
    blocks_data = json.load(f)

# Save combined CSV
output_file = "functions/CSV DATA/blocks_with_names.csv"
save_blocks_with_names(blocks_data, output_file)

import os
import pandas as pd

# Path to your CSV data directory
directory = 'functions/CSV DATA'

# Iterate through all files in the directory
for filename in os.listdir(directory):
    if filename.endswith('.csv'):
        file_path = os.path.join(directory, filename)
        
        # Load the CSV file
        df = pd.read_csv(file_path)
        
        # Rename columns: replace spaces with underscores
        df.columns = [col.replace(' ', '_') for col in df.columns]
        
        # Save the modified DataFrame back to the same file
        df.to_csv(file_path, index=False)
        print(f"Processed: {filename}")
import pandas as pd

# Load the CSV files
events_scores_path = 'functions/CSV DATA/all_event_scores.csv'
tournaments_path = 'functions/CSV DATA/tournaments.csv'
# Read the data
events_scores_df = pd.read_csv(events_scores_path)
tournaments_df = pd.read_csv(tournaments_path)
print(tournaments_df[tournaments_df["year"]==2025])

# Normalize column names for joining
tournaments_df.rename(columns={
    'event_name': 'Event_Name',
    'tour': 'Tour',
    'year': 'Year',
    'event_id': 'Event_ID',
    'player_name': 'Player_Name'
}, inplace=True)

# Perform the merge (inner join based on Event_ID and Player_Name)
merged_df = pd.merge(
    tournaments_df,
    events_scores_df,

    on=['Event_ID', 'Player_Name'],
    how='inner',
    suffixes=('', '_duplicate')  # Avoid suffixes where possible
)

# Remove duplicated columns explicitly
columns_to_keep = [col for col in merged_df.columns if not col.endswith('_duplicate')]
merged_df = merged_df[columns_to_keep]
columns_to_keep
# Drop specified columns
merged_df.drop(columns=[ 'last_fetched', 'Final_Position'], inplace=True)
# Extract 'Month' and 'Day' from 'Date' column
merged_df['Month'] = pd.to_datetime(merged_df['Date']).dt.month
merged_df['Day'] = pd.to_datetime(merged_df['Date']).dt.day
merged_df.drop(columns=['Date'], inplace=True)

# Rename ambiguous columns to make them more descriptive (if needed)
merged_df.rename(columns={
    'Year': 'Event_Year',
    'Tour': 'Event_Tour',
    'Event_Name': 'Event_Name_Final'
}, inplace=True)

# Reorder columns to place 'Month' and 'Day' after 'Event_Year'
columns_order = ['Event_ID', 'Player_Name', 'Event_Year', 'Month', 'Day'] + \
    [col for col in merged_df.columns if col not in ['Event_ID', 'Player_Name', 'Event_Year', 'Month', 'Day']]
merged_df = merged_df[columns_order]

# Save the cleaned dataframe
merged_df.to_csv("functions/CSV DATA/tours_events.csv", index=False)

import pandas as pd

# Load the datasets
players_info_path = 'functions/CSV DATA/players_info.csv'
tours_events_path = 'functions/CSV DATA/tours_events.csv'

players_info_df = pd.read_csv(players_info_path)
tours_events_df = pd.read_csv(tours_events_path)

# Harmonize column names for clarity
clear_column_names = {
    "sample_size": "SampleSize",
    "firstName": "FirstName",
    "lastName": "LastName",
    "countryCode": "CountryCode",
    "primary_tour": "PrimaryTour",
    "player_name": "PlayerName"
    # Add other renaming rules if needed
}

players_info_df.rename(columns=clear_column_names, inplace=True)

# Join the datasets on Player_Name/PlayerName
combined_df = pd.merge(
    tours_events_df,
    players_info_df,
    left_on="Player_Name",
    right_on="PlayerName",
    how="left"
)

# Save the combined dataframe to a CSV file
combined_df.to_csv("functions/CSV DATA/tours_events_players.csv", index=False)
file_path = "functions/CSV DATA/tours_events_players.csv"
df = pd.read_csv(file_path)

# Clean up column names
df.columns = [
    col.strip().replace(" ", "_").lower()  # Normalize column names
    for col in df.columns
]

# Save the cleaned DataFrame back to a CSV file if needed
# Drop columns starting with specific prefixes
prefixes_to_drop = ('100', '150', '50', 'under', 'over' , "playerskillratings")
columns_to_drop = [col for col in df.columns if col.startswith(prefixes_to_drop)]
df = df.drop(columns=columns_to_drop)

# Drop duplicates based on the specified columns
df = df.drop_duplicates(subset=["event_id", "player_name", "event_year", "month", "day", "event_name_final", "event_tour"], keep="first")

df['finish_date'] = pd.to_datetime(df['finish_date'], format="%d/%m/%Y", errors='coerce')

# Fill 'month' column with month extracted from 'finish_date'
df['month'] = df['finish_date'].dt.month

# Save the cleaned DataFrame
cleaned_file_path = "functions/CSV DATA/tours_events_players.csv"
df.to_csv(cleaned_file_path, index=False)

print(f"Cleaned CSV saved to {cleaned_file_path}")
print("combined tours_events_players is saved ")
df[df["event_year"]==2025]

CSV file 'functions/CSV DATA/tournaments.csv' has been created.
CSV file 'functions/CSV DATA/players_info.csv' has been created.
CSV file 'functions/CSV DATA/2025_tournaments_data.csv' created successfully.
Event ID: 1, year: 2022, date: 2022-06-11
Missing date for event ID: 10, year: 2020
Missing date for event ID: 10, year: 2021
Missing date for event ID: 10, year: 2022
Missing date for event ID: 10, year: 2023
Missing date for event ID: 10, year: 2024
Missing date for event ID: 100, year: 2021
Missing date for event ID: 100, year: 2022
Missing date for event ID: 100, year: 2023
Missing date for event ID: 100, year: 2024
Event ID: 104, year: 2020, date: 2020-02-09
Event ID: 104, year: 2022, date: 2022-02-13
Event ID: 104, year: 2023, date: 2023-02-12
Event ID: 104, year: 2024, date: 2024-02-11
Event ID: 107, year: 2024, date: 2024-03-10
Missing date for event ID: 11, year: 2020
Missing date for event ID: 11, year: 2021
Missing date for event ID: 11, year: 2022
Missing date for event 

/var/folders/pr/m_s6klxj7ls96t87l23xgx380000gn/T/ipykernel_41596/1374968907.py:343: DtypeWarning: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(file_path)


Processed: tours_events_players_cleaned.csv
Processed: players_info.csv
       event_id  year      player_name  sg_total  perkcup_points  \
0            16  2025  Horschel, Billy       0.0            10.0   
1            16  2025   Garnett, Brice       0.0            10.0   
2            16  2025      Finau, Tony       0.0            25.0   
3            16  2025      Kirk, Chris       0.0            10.0   
4            16  2025   Campos, Rafael       0.0            10.0   
...         ...   ...              ...       ...             ...   
28953         6  2025              NaN       NaN             NaN   
28954        60  2025              NaN       NaN             NaN   
28955         7  2025              NaN       NaN             NaN   
28956        88  2025              NaN       NaN             NaN   
28957         9  2025              NaN       NaN             NaN   

      finish_position tour finish_date  \
0                  51  NaN         NaN   
1                  48  NaN 

,event_id,player_name,event_year,month,day,sg_total_x,perkcup_points,finish_position,event_tour,finish_date,...,owgr_change,sg_putt_rank,sg_ott_pred,sg_total_pred,sg_ott_rank,accuracy_rank,datagolfindex,sg_app_pred,perkcuppoints2024_2,act1
0,16,"Horschel, Billy",2025,NaN,NaN,0.0,10.0,51,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,16,"Finau, Tony",2025,NaN,NaN,0.0,25.0,15,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,16,"Kirk, Chris",2025,NaN,NaN,0.0,10.0,44,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,16,"Taylor, Nick",2025,NaN,NaN,0.0,10.0,48,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,16,"Matsuyama, Hideki",2025,NaN,NaN,0.0,100.0,1,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
709,6,"List, Luke",2025,NaN,NaN,0.0,10.0,72,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
712,6,"Streelman, Kevin",2025,NaN,NaN,0.0,10.0,53,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
714,6,"Snedeker, Brandt",2025,NaN,NaN,0.0,10.0,53,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
718,6,"Harman, Brian",2025,NaN,NaN,0.0,25.0,21,NaN,NaT,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df[df["event_year"]==2025]

In [ ]:
import json
import csv

# Load JSON data from file
with open("firestore_full_database.json", "r") as file:
    data = json.load(file)

# Extract relevant tournament data
tournaments = data.get("2025_tournaments", {})

tournament_rows = []

for tournament_id, tournament_data in tournaments.items():
    players = tournament_data.get("players", {})
    for player_id, player_data in players.items():
        tournament_rows.append([
            tournament_id,
            2025,
            player_data.get("player_name", ""),
            player_data.get("sg_total", 0),
            player_data.get("perkcup_points", 0),
            player_data.get("fin_text", "")
        ])
# Save to CSV
csv_filename = "functions/CSV DATA/2025_tournaments_data.csv"
with open(csv_filename, "w", newline="") as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(["event_id", "year","player_name", "sg_total", "perkcup_points", "finish_position"])
    csv_writer.writerows(tournament_rows)

print(f"CSV file '{csv_filename}' created successfully.")
# Load both CSV files
df1 = pd.read_csv('functions/CSV DATA/2025_tournaments_data.csv')
df2 = pd.read_csv('functions/CSV DATA/tournaments.csv')
# Concatenate while keeping all columns from both DataFrames

combined_df = pd.concat([df1, df2], axis=0, ignore_index=True)
id_to_name = {
    6: "Sony Open in Hawaii",
    2: "The American Express",
    16: "The Sentry"
}

# Fill missing event_name values based on event_id
combined_df['event_name'] = combined_df.apply(lambda row: id_to_name.get(row['event_id'], row['event_name']), axis=1)

# Save the merged DataFrame to a new CSV file
combined_df.to_csv('functions/CSV DATA/tournaments.csv', index=False)
print("CSV files have been successfully combined and saved as 'combined_tournaments.csv'.")

51
48
15
44
57
48
40
4
1
15
29
26
5
32
30
32
53
15
8
36
3
15
26
5
24
13
13
44
40
15
8
36
56
26
8
46
WD
30
48
40
52
2
32
36
15
24
15
5
32
8
15
8
36
46
55
53
15
58
40
CU
CU
CU
21
CU
34
43
CU
CU
34
WD
CU
CU
CU
21
12
CU
CU
CU
CU
CU
CU
2
34
CU
43
51
CU
CU
CU
CU
34
5
12
58
29
58
CU
70
12
CU
21
43
58
21
CU
CU
CU
1
29
43
CU
21
CU
3
18
CU
CU
18
11
51
18
CU
43
CU
12
WD
29
CU
CU
CU
CU
CU
34
CU
CU
12
12
68
CU
CU
58
7
64
29
CU
43
CU
7
CU
CU
CU
58
CU
CU
CU
29
CU
CU
21
CU
CU
CU
7
CU
CU
34
CU
CU
58
51
64
CU
CU
CU
66
CU
CU
CU
21
51
CU
CU
CU
43
CU
CU
34
CU
68
34
66
CU
CU
21
51
CU
51
CU
CU
5
43
CU
CU
CU
CU
CU
34
CU
CU
71
CU
51
CU
7
3
CU
CU
37
16
CU
37
30
37
CU
CU
16
CU
CU
1
CU
CU
16
6
59
30
59
CU
53
CU
30
CU
CU
10
CU
30
CU
53
CU
21
CU
45
CU
45
CU
21
10
3
53
21
CU
30
69
70
30
3
CU
30
CU
CU
37
45
CU
45
16
WD
45
CU
6
CU
CU
CU
16
CU
CU
21
45
CU
5
WD
CU
37
CU
45
37
59
65
2
37
53
CU
70
CU
59
CU
CU
CU
CU
72
65
CU
45
CU
6
10
CU
65
59
72
CU
CU
CU
10
CU
CU
10
CU
72
CU
CU
CU
CU
CU
CU
21
10
CU
65
6
CU
CU
37
CU
CU
CU

In [10]:
import pandas as pd
import json

# Step 1: Load the JSON file for blocks_collection
blocks_file_path = 'blocks_collection.json'
with open(blocks_file_path, 'r') as f:
    blocks_data = json.load(f)

# Step 2: Extract PerkCup Points data
perkcuppoints_data = []
for block_key, block_data in blocks_data.items():
    for id_key, player_data in block_data.get("perkcup_points", {}).items():
        for year, points in player_data.items():
            year_cleaned = year  # Keep the year as-is
            perkcuppoints_data.append({"Block": block_key, "PlayerID": id_key, "Year": year_cleaned, "PerkCupPoints": points})

# Convert to DataFrame and pivot years into columns
perkcuppoints_df = pd.DataFrame(perkcuppoints_data)
perkcuppoints_pivot = perkcuppoints_df.pivot(index=["Block", "PlayerID"], columns="Year", values="PerkCupPoints").reset_index()

# Step 3: Extract Players' Historical Finish Positions
players_data = []
for block_key, block_data in blocks_data.items():
    for player_name, player_details in block_data.get("players", {}).items():
        for year_tournament, position in player_details.get("historicalFinishPositions", {}).items():
            year, tournament = map(str, year_tournament.split(","))
            players_data.append({
                "Block": block_key,
                "PlayerName": player_name,
                "Year": year,
                "Tournament": tournament,
                "Position": position,
                "Participations": player_details.get("participations", 0),
                "TotalFieldPoints": round(player_details.get("currentblocktotalfieldpoints", 0), 2)  # Format to 2 decimal places
            })

# Convert to DataFrame
players_df = pd.DataFrame(players_data)

# Step 4: Load the JSON file for firestore_full_database
firestore_file_path = 'firestore_full_database.json'
with open(firestore_file_path, 'r') as f:
    firestore_data = json.load(f)

# Step 5: Extract player IDs and names
players_ids_data = []
for player_id, player_info in firestore_data.get("players", {}).items():
    players_ids_data.append({
        "PlayerID": player_id,
        "PlayerName": player_info.get("player_name", "Unknown")
    })

# Convert to DataFrame
players_ids_df = pd.DataFrame(players_ids_data)

# Step 6: Merge PerkCup Points with player IDs and names
combined_perkcuppoints_df = pd.merge(perkcuppoints_pivot, players_ids_df, on="PlayerID", how="left")

# Step 7: Merge the combined PerkCup Points with players' data on PlayerName
final_combined_df = pd.merge(players_df, combined_perkcuppoints_df, on="PlayerName", how="left")

# Step 8: Save results to a CSV file
final_combined_csv_path = 'functions/CSV DATA/blocks_with_names.csv'
final_combined_df.to_csv(final_combined_csv_path, index=False)

# Output file path
print("Final Combined CSV saved as:", final_combined_csv_path)

KeyError: 'event_id'

# Tests

In [49]:
import requests

response = requests.post('http://localhost:5000/query', json={
    'question': 'what are the best players in terms of winning  tournamants  ?',
    'theme': 'tournament'  # or 'blocks' or 'internet'
})

results = response.json()
print(results["analysis_details"])
print(results["response"])

> Entering new AgentExecutor chain...

💭 Reasoning:
To determine the best players in terms of winning tournaments, I should analyze the `finish_position` column, as it indicates how well a player performed in a tournament. The lower the `finish_position`, the better the player performed. I will calculate the average `finish_position` for each player and rank them accordingly.

🔧 Step:
Action: python_repl_ast
Action Input: df.groupby('player_name')['finish_position'].mean().sort_values()TypeError: agg function failed [how->mean,dtype->object] It seems that the `finish_position` column has some non-numeric values, which is causing the error. I will first check the data types of the columns to confirm this.

🔧 Step:
Action: python_repl_ast
Action Input: df.dtypesevent_id                              int64
player_name                          object
event_year                            int64
month                                 int64
day                                   int64
          

In [ ]:
import requests
import csv

# List of questions with themes
questions = [
    {
        "question": " give me all the tournaments ",
        "theme": "tournament"
    }
    # Add additional questions as needed
]

# API endpoint
api_url = 'http://localhost:5000/query'

# File to save results
csv_file = 'results.csv'

# Open the CSV file and write the headers
with open(csv_file, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Question', 'Theme', 'Analysis Details', 'Response'])

    # Loop through questions and send API requests
    for q in questions:
        try:
            response = requests.post(api_url, json=q)
            response.raise_for_status()  # Check for HTTP request errors
            data = response.json()
            # Write the result to the CSV
            writer.writerow([q['question'], q['theme'], data.get('results', ''), data.get('response', '')])
            print(f"Question processed: {q['question']}")
        except requests.exceptions.RequestException as e:
            print(f"Failed to process question: {q['question']} - Error: {e}")

print(f"Results saved to {csv_file}")
results = pd.read_csv('results.csv')
print(results["Analysis Details"][0])
print(results["Response"][0])



Question processed:  give me all the tournaments 
Results saved to results.csv
> Entering new AgentExecutor chain...

💭 Reasoning:
To find all the tournaments, I need to extract the unique values from the 'event_name_final' column in the dataframe. This will give me a list of all the tournaments present in the dataset.

🔧 Step:
Action: python_repl_ast
Action Input: df['event_name_final'].unique()['London' 'The Honda Classic' 'Tucson'
 'Cognizant Classic in The Palm Beaches' 'AdventHealth Championship'
 'Country Club de Bogota Championship'
 'Astara Golf Championship presented by Mastercard'
 'Astara Chile Classic presented by Scotiabank' 'THE PLAYERS Championship'
 'Visit Knoxville Open' 'Orlando'
 'WinCo Foods Portland Open presented by KraftHeinz' 'RBC Heritage'
 'Adelaide' 'Simmons Bank Open for the Snedeker Foundation'
 'Lincoln Land Championship presented by LRS'
 'Memorial Health Championship presented by LRS'
 'Pinnacle Bank Championship presented by Aetna'
 'Pinnacle Bank Champ

# TEST